### Variable Stoichiometry and Nitrogen Fixation
--- 

In this notebook I will write simulations to explore the role of deviations from Redfield on the nutrient stoichiometry of the ocean and the biota. The starting point will be Toby Tirrel's two box ocean model which predicts that nitrogen fixers cause the ocean to converge to the Redfield ratio. We will initially study the implications of allowing different N:P ratios in Nitrogen Fixers and phytoplankton, and then we will explore increasing the number of boxes.


In [1]:

from scipy.integrate import ode

%pylab inline

import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
matplotlib.use("Agg")
import scipy.io as sio
import scipy.signal
from pylab import *

asin=np.arcsin
sqrt=np.sqrt
sin=np.sin
cos=np.cos
pi=np.pi
exp=np.exp
tan=np.tan
I=1j
inf=np.inf

hilbert=scipy.signal.hilbert


Populating the interactive namespace from numpy and matplotlib


/usr/lib/pymodules/python2.7/matplotlib/__init__.py:1173: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
### Here are important constants

SD = 500.0   # Surface layer depth in meters
DD = 3230.0  # Deep layer depth in meters
K = 3.0 # Mixing coefficient in meters/year
SR = .95 # Fraction of TPP regenerated above 500m
SF = .000 # Fraction of TPP permanently incorporated into sediments
DR = .05 # Fraction of TPP regenerated below 500m
DN = .015 # Fraction of TPP of N that is regenerated via denitrification to N2
RP = 2.0e-4 # River input of dissolved P in mol P/m^2/year
RN = 6.0e-3 # River input of dissolved N in mol N/m^2/year
AN = 7.5e-3 # Atmospheric input of N in molN/m^2/year
RPhyto = 16.0 # N:P ratio in organic matter from phytoplankton
RNFixers = 16.0 # N:P ratio in Nitrogen Fixers
muNF = 87.6 # Maximum growth rate of nitrogen fixing bacteria in units of 1/year
muPhyto = 91.25 # Maximum growth rate of phytoplankton in units of 1/year
PH = 3.0e-5 # Half saturation constant for Phosphate in mol P /m^3
NH = 5.0e-4   # Half saturation constant for Nitrate in mol N /m^3
M = 73.0    # Mortality in 1/year


### Next are the functions that define the right hand side of the system of equations

# We first define the dynamical variables:

# NFixer stands for the population of nitrogen fixers
# PhytoP stands to phytoplankton
# PS stands for the surface phosphate
# NS stands for surface nitrate
# ND stands for deep nitrate
# PD stands for deep phosphate


###

def dNFixer(NFixer,PS):
    return muNF*PS/(PS+PH)*NFixer - M*NFixer

def dPhytoP(PhytoP,PS,NS):
    return muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP - M*PhytoP

def dPS(PS,NS,PD,ND,PhytoP,NFixer):
    return -muNF*PS/(PS+PH)*NFixer/RNFixers+M*(SR)*NFixer/RNFixers+M*(SR)*PhytoP/RPhyto-muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP/RPhyto+K*(PD-PS)/SD+RP/SD  

def dNS(PS,NS,PD,ND,PhytoP,NFixer):
    return -muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP + M*(SR-0.75*DN)*NFixer + M*(SR-0.75*DN)*PhytoP +K*(ND-NS)/SD+(RN+AN)/SD

def dPD(PS,NS,PD,ND,PhytoP,NFixer):
    return M*DR*NFixer/RNFixers*SD/DD+M*DR*PhytoP/RPhyto*SD/DD - K*(PD-PS)/DD

def dND(PS,NS,PD,ND,PhytoP,NFixer):
    return M*(DR-0.25*DN)*NFixer*SD/DD+M*(DR-0.25*DN)*PhytoP*SD/DD-K*(ND-NS)/DD


def dState(t,state):
    PS=state[0]
    NS=state[1]
    PD=state[2]
    ND=state[3]
    PhytoP=state[4]
    NFixer=state[5]
    
    dS = np.zeros(6)
    dS[0] = dPS(PS,NS,PD,ND,PhytoP,NFixer)
    dS[1] = dNS(PS,NS,PD,ND,PhytoP,NFixer)
    dS[2] = dPD(PS,NS,PD,ND,PhytoP,NFixer)
    dS[3] = dND(PS,NS,PD,ND,PhytoP,NFixer)
    dS[4] = dPhytoP(PhytoP,PS,NS)
    dS[5] = dNFixer(NFixer,PS)
    return dS

###










In [3]:
### In this box we will define functions that determine the equilibrium state of the system


PSEquilibrium = M*PH/(muNF-M)
NSEquilibrium = M*NH/(muPhyto-M)

PhytoPEquilibrium = 1.0/(M*SD)*(RN+AN+RP*RNFixers*(1-SF-DN)/SF)/(SF+DN+RNFixers/RPhyto*(1-SF-DN))
NFixerEquilibrium = 1.0/(M*SD)*(RN+AN-RPhyto*RP*(SF+DN)/SF)/(DN+SF-1-RPhyto/RNFixers*(SF+DN))

PDEquilibrium = PSEquilibrium+M*DR*SD/K*(NFixerEquilibrium/RNFixers+PhytoPEquilibrium/RPhyto)
NDEquilibrium = NSEquilibrium+M*(DR-0.25*DN)*SD/K*(NFixerEquilibrium+PhytoPEquilibrium)

RS = NSEquilibrium/PSEquilibrium
RD = NDEquilibrium/PDEquilibrium



ZeroDivisionError: float division by zero

In [13]:
dState(0,vecStart)

array([ -8.00442458e-08,   2.37169225e-19,   1.18951201e-08,
         1.01643954e-20,   0.00000000e+00,   0.00000000e+00])

In [286]:
dPhytoP(PhytoPEquilibrium,PSEquilibrium,NSEquilibrium)

0.0

In [287]:
PhytoPEquilibrium

4.380518791711463e-05

In [288]:
(NFixerEquilibrium*RNFixers+PhytoPEquilibrium*RPhyto)/(NFixerEquilibrium+PhytoPEquilibrium)

17.59214293960983

In [15]:
.0257/.00175

14.685714285714285

In [14]:
vecStart


[0.00015000000000000007,
 0.002,
 0.0017500000000000003,
 0.02578890442001687,
 4.380518791711463e-05,
 3.8130740680805543e-07]

In [9]:
vecStart=[PSEquilibrium,NSEquilibrium,PDEquilibrium,NDEquilibrium,PhytoPEquilibrium,NFixerEquilibrium]


rr = ode(dState).set_integrator('dopri5')
rr.set_initial_value(vecStart,0)
rr.t
t1 = 100
dt = 1e-1
while (rr.successful() and rr.t < t1):
    rr.integrate(rr.t+dt)
    
    
    
    





In [10]:
rr.y

array([  1.50000000e-04,   2.00000000e-03,   1.75000000e-03,
         2.57889044e-02,   4.38051879e-05,   3.81307407e-07])

In [134]:
# Alternate equilibrium to deal with bs

PSAlt = (PH/(muNF/M-1))
NSAlt = (NH/(muPhyto/M-1))

PhytoAlt = (1/(M*SD))*(RP*RPhyto*(1-SF-DN)/SF+RN+AN)
NFixerAlt = (1/(M*SD))*(RP*RPhyto*(SF+DN)/SF-RN-AN)

PDAlt = PSAlt +(RP*(1-SR-SF)/(K*SF))
PDAAlt = PSAlt+M*DR*SD/K*(PhytoAlt/RPhyto+NFixerAlt/RPhyto)
NDAlt = NSAlt + (RP*RPhyto*(1-SR-SF-0.25*DN)/(K*SF))

altVec = np.array([PSAlt,NSAlt,PDAlt,NDAlt,PhytoAlt,NFixerAlt])






In [154]:
altVec-vecStart


array([ -2.71050543e-20,   0.00000000e+00,   1.30104261e-18,
         2.08166817e-17,   0.00000000e+00,   5.29395592e-23])

In [123]:
(RP*(1-SR-SF)/(K*SF))-M*DR*SD/K*(PhytoAlt/RPhyto+NFixerAlt/RPhyto)

2.714560000000141e-05

In [124]:
M*DR*SD/K*(PhytoAlt/RPhyto+NFixerAlt/RPhyto)

0.0015728544000000002

In [130]:
PhytoAlt/RPhyto+NFixerAlt/RPhyto-(RP/(M*SD*SF))

-4.648219178082185e-08

In [132]:
RP/(M*SD*SF)

2.7397260273972604e-06

In [133]:
PhytoAlt+NFixerAlt-((1/(M*SD))*(RP*RPhyto*(1-SF-DN)/SF+RN+AN)+(1/(M*SD))*(RP*RPhyto*(SF+DN)-RN-AN))

0.0

In [138]:
1.0/(M*SD)*(RN+AN+RP*(1-SF-DN)/SF)/(SF+DN+RNFixers/RPhyto*(1-SF-DN))

3.063013698630137e-06

In [139]:
PhytoAlt

4.346027397260274e-05

In [141]:
(SF+DN+RNFixers/RPhyto*(1-SF-DN))

1.0

In [149]:
1.0/(M*SD)*(RN+AN+RP*RPhyto*(1-SF-DN)/SF)

4.346027397260274e-05

In [144]:
(1/(M*SD))*(RP*RPhyto*(1-SF-DN)/SF+RN+AN)

4.346027397260274e-05

In [147]:
(RN+AN+RP*(1-SF-DN)/SF)

0.1118

In [148]:
(RP*RPhyto*(1-SF-DN)/SF+RN+AN)

1.5863